How can we guess consecutive letters in a message, following the game presented in Shannon (1952), "There is no reverse on a motorcycle..." 

In [14]:
import sys, time, random, pdb, srilm, pandas

Load the SRILM language models. Type-weighted has transitions computed over 25k types. Token-weighted takes the token frequencies in a corpus into account, e.g. the OPUS subtitle corpus. Utterance token model treats each utterance as a separate tokens (e.g., to take word-to-word dependencies into account). Lexical looks at the token probabilities in the language.

In [2]:
modelStore = {}
modelStore['typeModel'] = srilm.LM("/home/stephan/python/shannonGame/englishCharacterModel.LM", lower=True)
modelStore['tokenModel'] = srilm.LM("/shared_ssd/VoB/EnglishTokenSSmodel/opus_letterized.LM", lower=True)
modelStore['utterance_tokenModel'] = srilm.LM("/shared_ssd/VoB/EnglishTokenSSmodel/opus_utterance_letterized.LM", lower=True)
modelStore['lexModel'] = srilm.LM('/shared_hd0/corpora/BNC/SRILM/English.lm.KNN', lower=True) #this LM is much larger than the others

In [3]:
def getRankedContinuation(lm, preceding):	
    # get a ranked set of continuations along with their probabilities
	preceding =  [x for x in preceding if x != ''] # remove padding
	context = [lm.vocab.intern(w) for w in preceding]
	best_idx = None
	best_logprob = -1e100
	continuations = []	
	for i in xrange(lm.vocab.max_interned() + 1):		
		logprob = lm.logprob(i, context)
		continuations.append({'word':lm.vocab.extern(i), 'prob':logprob})
	continuationsDF = pandas.DataFrame(continuations)
	return(continuationsDF.sort(['prob'], ascending=False))

In [4]:
def shannonGame(target,modelStore,model, guessTime, showStrings):	
    #target is the string to guess
    #model is the path to the LM from pysrim
    #guessTime is an integger specifying how many ms to sleep between guesses (for a live demo)
    #showStrings is a boolean specifying whether progress should be displayed
    #!!!this needs to be refactored.
	phraseToGuess = list(target)
	numGuesses = 0
	letters = list('abcdefghijklmnopqrstuvwxyz')

	if model == 'uniformCharacterProb':
		#Guess each letter with equal probability
		correctString = ''
		for letter in phraseToGuess:	
			if letter in (' ','.'):
				#write spaces and punctuation immediately
				correctString = correctString + letter		
				continue
			else:			
				random.shuffle(letters)
				for guess in letters:
					numGuesses += 1
					if showStrings:
						print(correctString + guess)
					time.sleep(guessTime)			
					if guess == letter:
						correctString = correctString + letter
						break

	elif model == 'uniphoneCharacterProb':					
		#Guess each letter with probability in proportion to types in language
		uniphoneModel = getRankedContinuation(modelStore['typeModel'],[])
		letterDF = pandas.DataFrame(list('abcdefghijklmnopqrstuvwxyz'), columns=['word'])
		letters = letterDF.merge(uniphoneModel).sort(['prob'], ascending=False)['word']

		correctString = ''
		for letter in phraseToGuess:	
			if letter in (' ','.'):
				#write spaces and punctuation immediately
				correctString = correctString + letter		
				continue
			else:			
				for guess in letters:
					numGuesses += 1
					if showStrings:
						print(correctString + guess)
					time.sleep(guessTime)			
					if guess == letter:
						correctString = correctString + letter
						break
	elif model == 'fivephoneCharacterProb':					
		#Guess each letter with probability according to phone transitions in the language (still type-based)
		letterDF = pandas.DataFrame(list('abcdefghijklmnopqrstuvwxyz'), columns=['word'])
		correctString = ''
		cachedWord = []
		for letter in phraseToGuess:	
			if letter in (' ','.'):
				#write spaces and punctuation immediately
				correctString = correctString + letter		
				cachedWord = [] #restart the cached word
				continue
			else:	
				continuations = getRankedContinuation(modelStore['typeModel'],cachedWord[::-1])		
				lettersDF_local = letterDF.copy(deep=True)	

				for guess in lettersDF_local.merge(continuations).sort(['prob'], ascending=False)['word']:
					numGuesses += 1
					if showStrings:
						print(correctString + guess)
					time.sleep(guessTime)			
					if guess == letter:
						correctString = correctString+letter
						cachedWord.append(letter)
						break
	elif model == 'fivephoneCharacterProb_token':					
		#Guess each letter with probability according to phone transitions in the language (token based)

		letterDF = pandas.DataFrame(list('abcdefghijklmnopqrstuvwxyz'), columns=['word'])

		correctString = ''
		cachedWord = []
		for letter in phraseToGuess:	
			if letter in (' ','.'):
				#write spaces and punctuation immediately
				correctString = correctString + letter		
				cachedWord = [] #restart the cached word
				continue
			else:	
				continuations = getRankedContinuation(modelStore['tokenModel'],cachedWord[::-1])		
				lettersDF_local = letterDF.copy(deep=True)	

				for guess in lettersDF_local.merge(continuations).sort(['prob'], ascending=False)['word']:
					numGuesses += 1
					if showStrings:
						print(correctString + guess)
					time.sleep(guessTime)			
					if guess == letter:
						correctString = correctString+letter
						cachedWord.append(letter)
						break
	elif model == 'fivephoneCharacterProb_token_utterance':						
		#Guess each letter with probability according to phone transitions in the language (token based, built on utterances)
		letterDF = pandas.DataFrame(list('abcdefghijklmnopqrstuvwxyz'), columns=['word'])

		correctString = ''
		cachedPrevious = []
		for letter in phraseToGuess:	
			if letter in (' ','.'):
				#write spaces and punctuation immediately
				correctString = correctString + letter		
				continue
			else:	
				continuations = getRankedContinuation(modelStore['tokenModel'],cachedPrevious[::-1])		
				lettersDF_local = letterDF.copy(deep=True)	

				for guess in lettersDF_local.merge(continuations).sort(['prob'], ascending=False)['word']:
					numGuesses += 1
					if showStrings:
						print(correctString + guess)
					time.sleep(guessTime)			
					if guess == letter:
						correctString = correctString+letter
						cachedPrevious.append(letter)

						if len(cachedPrevious) > 4: #trim it
							cachedPrevious = cachedPrevious[-4:]
						break
	elif model == 'lexical':
		#Guess each letter by marginalizing over phone probabilities for the next lexical item
		letterDF = pandas.DataFrame(list('abcdefghijklmnopqrstuvwxyz'), columns=['word'])

		correctString = ''
		cachedPreviousWords = []
		cachedPreviousPhones = []
		for letter in phraseToGuess:	
			if letter in (' ','.'):
				#write spaces and punctuation immediately
				correctString = correctString + letter	
				cachedPreviousWords.append(''.join(cachedPreviousPhones))
				cachedPreviousPhones = []
				if len(cachedPreviousWords) > 2: #trim it to 2 words
					cachedPreviousWords = cachedPreviousWords[-2:]
			else:	
				allContinuations = getRankedContinuation(modelStore['lexModel'],cachedPreviousWords[::-1])
				#filter
				continuations = allContinuations[allContinuations['word'].str.startswith("".join(cachedPreviousPhones))]
				continuations['shortWord'] = [x[len(cachedPreviousPhones):] for x in continuations['word']]
				continuations['stringLength'] = [len(x) for x in continuations['shortWord']]
				#subset to items that have an initial letter
				continuations = continuations[continuations['stringLength'] > 0]

				continuations['initialLetter'] = [list(x)[0] for x in continuations['shortWord']]
				continuations['posProb'] = 10.**continuations['prob']
				initLetter = continuations.groupby(by=['initialLetter'])['posProb'].agg([sum])
				initLetter['sum'] = initLetter['sum'] / sum(initLetter['sum'])
				initLetter['word'] = initLetter.index
								
				lettersDF_local = letterDF.copy(deep=True)	
				for guess in lettersDF_local.merge(initLetter).sort(['sum'], ascending=False)['word']:
					numGuesses += 1
					if showStrings:
						print(correctString + guess)
					time.sleep(guessTime)			
					if guess == letter:
						correctString = correctString+letter
						cachedPreviousPhones.append(letter)					
						break
	return({'model':model, 'numGuesss':numGuesses}) 

In [5]:
target = ("there is no reverse on a motorcycle") 
           #"pelicans are the least trustworthy of all the birds that fly in the sky"
           #"kale is a nutritious addition to any meal"

# Single Core

In [13]:
#
print shannonGame(target, modelStore, 'uniformCharacterProb', guessTime=0, showStrings=False)
print shannonGame(target, modelStore, 'uniformCharacterProb', guessTime=0, showStrings=False)
print shannonGame(target, modelStore, 'fivephoneCharacterProb', guessTime=0, showStrings=False)
print shannonGame(target, modelStore, 'fivephoneCharacterProb_token_utterance', guessTime=0, showStrings=False)
print shannonGame(target, modelStore, 'lexical', guessTime=0, showStrings=False)


{'model': 'uniformCharacterProb', 'numGuesss': 384}
{'model': 'uniformCharacterProb', 'numGuesss': 338}
{'model': 'fivephoneCharacterProb', 'numGuesss': 116}
{'model': 'fivephoneCharacterProb_token_utterance', 'numGuesss': 136}
{'model': 'lexical', 'numGuesss': 75}
